imports the necessary modules needed.

In [1]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
import katpoint
import numpy as np
import time as time
import pandas as pd
import math

We compute the rise and set time of the Sun for the given timestamps. This helps us to define the day and the night time. For the purpose of this script we are only interested in identifying the night time for every timestamps. We plot the LST against the date to show the rise and set of the Sun.

In [13]:

def rise_set(timestamp):
    site = katpoint.Antenna("m000, -30:42:39.8, 21:26:38.0, 1086.6, 13.5") # the site position using one antenna(m000)
    site.observer.date = katpoint.Timestamp(timestamp).to_ephem_date() # Set time
    site.observer.horizon='00:00:00.0' # Set horizon
    observer = site.observer 
    Sun = katpoint.Target("Sun,special")# Make sun object
    ephem_sun = Sun.body #Sun as an object
    ephem_sun.compute(observer) #compute for position of the observer
    return observer.next_rising(ephem_sun),observer.next_setting(ephem_sun)

site = katpoint.Antenna("m000, -30:42:39.8, 21:26:38.0, 1086.6, 13.5")
settime,risetime = np.zeros((31,2)),np.zeros((31,2))

for day in range(31):

    timestamp = katpoint.Timestamp() + 24*60*60*day
    #print(timestamp)
    rise_time,set_time = rise_set(timestamp)
    #print
    risetime[day] =katpoint.Timestamp(rise_time),np.degrees(site.local_sidereal_time(rise_time))/15.
    settime[day] =katpoint.Timestamp(set_time),np.degrees(site.local_sidereal_time(set_time))/15.
    # Array values are unix time and decimal LST time in hours


# fig = plt.figure(figsize=(14,7))
# plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d/%m/%Y'))
# plt.gca().xaxis.set_major_locator(mdates.DayLocator())

# dates=[dt.datetime.fromtimestamp(ts) for ts in risetime[:,1]]
# plt.plot(risetime[:,1],'go',label="Rising")
# dates2=[dt.datetime.fromtimestamp(ts) for ts in settime[:,1]]
# plt.plot(settime[:,1],'ro',label="Setting")

# plt.title(" Source Rise and Set ")
# plt.ylabel('LST (Hours)')
# plt.xlabel('Date')
# plt.gcf().autofmt_xdate()
# plt.grid()
# plt.legend()


None
None


Identifying the night by taking the LST set and rise for each timestamp. 

In [20]:
t = '2023-05-01 20:18:47.830'
katpoint.Timestamp(t).to_ephem_date()

45046.346386921294

In [31]:
site = katpoint.Antenna("m000, -30:42:39.8, 21:26:38.0, 1086.6, 13.5") # the site position using one antenna(m000)
site.observer.date = katpoint.Timestamp(timestamp).to_ephem_date() # Set time
site.observer.horizon='00:00:00.0' # Set horizon
#print(site)
observer = site.observer
print(observer)
Sun = katpoint.Target("Sun,special")# Make sun object
print(Sun)
ephem_sun = Sun.body #Sun as an object
print(ephem_sun)
#ephem_sun.compute(observer) #compute for position of the observer

observer.next_rising(ephem_sun), observer.next_setting(ephem_sun)

<ephem.Observer date='2023/5/31 20:24:46' epoch='2000/1/1 12:00:00' lon='21:26:38.0' lat='-30:42:39.8' elevation=1086.6m horizon=0:00:00.0 temp=15.0C pressure=0.0mBar>
Sun, tags=special, no flux info
<Sun "Sun" at 0x7f8c7a98f048>


(45076.72657534877, 45077.151147340104)

In [28]:
observer.previous_rising(ephem_sun)

45075.7261972573

In [25]:
print(time.ctime(45076.78341865581))

Thu Jan  1 14:31:16 1970


In [12]:
def nighttime(riset,sett):
    'Arranging LST rise and set time to show night time '
    ind1 = 0
    ind2 = -1
       
    rt_new = np.delete(riset, ind1)
    st_new = np.delete(sett,ind2)
    setrise = {}
    for i in range(len(riset)-1):
        setrise[i] = [st_new[i], rt_new[i]]
    return setrise

Given csv file constaining observational information for observation schedule blocks, the function extract the night time observations only. 

In [13]:
def nightobs(filename):
    'Extracting only night time observations\
    from a given file of observational information'
    datefile = dates
    ndates = []
    for j in range(len(datefile)):
        ndates.append(datefile[j].strftime('%Y-%m-%d'))
    data = pd.read_csv(filename)
    night_obs = data['night_obs']
    LST_starts = data['lst_start']
    LST_ends = data['lst_start_end']
    duration = data['simulated_duration']
    proposal_id = data['proposal_id']
    #proposal_id = sorted(proposal_id)
    ID = data['id']
    
    obs = {}
    for i in range(len(night_obs)):
        if night_obs[i] == 'Yes':
            obs[ID[i]] = [proposal_id[i],LST_starts[i],LST_ends[i],duration[i]]
    return obs, ndates

Pass a csv file in order to run parameter_extract function. 

In [15]:

d = 'all_night_obs.csv'

def parameter_extract(datafile):
    '''extracts 4 parameters from a datafile and converts
    duration from seconds to hours, and observations LST time to match with the 
    LST for night time
    
    parameters: 
    ---------------
    datafile : a 
    
    '''
    obs , date = nightobs(datafile)
    duration_ = []
    LST_start = []
    LST_end = []
    obs_id = []
    for i in obs.values():
        duration_.append(round(i[3]/3600,2))
        g1 = i[1].replace(':', '.')
        f1 = float(g1)
        LST_start.append(f1)
        g2 = i[2].replace(':', '.') 
        f2 = float(g2)
        LST_end.append(f2)
        obs_id.append(i[0])
    duration_ = [0 if math.isnan(x) else x for x in duration_]
    
    return LST_start, LST_end, duration_, obs_id, date
# print(LST_start)
# print(LST_end)
# print(duration_)
# print(obs_id)

LST_start, LST_end, duration_, obs_id, date = parameter_extract(d)

In [6]:
rt = risetime[:,1]
st = settime[:,1]
SetRise = nighttime(rt,st)

#print(SetRise)


#SetRise.values()

In [16]:
def obs_selection(s_set, s_rise, start_r, end_r, dur, observation_id):
    """ Check if the observation time range is within the LST time range.
    
    Parmeters:
    -------------
    s_set : float
        LST sun set
    s_rise : float
        LST sun rise
    start_r : float
        observation allowed start LST range
    end_r : float
        observation allowed end LST range
    dur : float 
        observation duration in hours
    """
    # check if observation start or end time is within the LST range
    cond1 = start_r >= s_set
    cond2 = end_r > s_set
    if  cond1 or cond2:
        if (cond1 and cond2) or cond1:
            start_time = start_r
            end_time = start_time + dur
        else:
            start_time = end_r
            end_time = start_time + dur
        if end_time > s_rise:
            return print('{} observation end time is outside the LST range'.format(observation_id))
        else:
            if end_time >= 24:
                end_time = end_time - 24
    else:
        return print('{} both start and the end time are outside the LST range'.format(observation_id))
    
    
    return print('Start time for {} is {} and end_time is {}'.format(observation_id, start_time, end_time))
    

In [17]:
ss = [i[0] for i in SetRise.values()] 
sr = [i[1] for i in SetRise.values()] 
a = 0
for ind in range(len(ss)):
    print("Date : {} ------------------------------------------------".format(date[ind]))
    for ind2 in range(len(LST_start)):
        d = obs_selection(ss[ind], sr[ind], LST_start[ind2], LST_end[ind2], duration_[ind2], obs_id[ind2])

Date : 2023-02-21 ------------------------------------------------
SCI-20220822-DP-01 observation end time is outside the LST range
SCI-20220822-KK-01 observation end time is outside the LST range
SCI-20220822-CV-01 observation end time is outside the LST range
SCI-20220822-TJ-01 observation end time is outside the LST range
SCI-20220822-CV-01 observation end time is outside the LST range
SCI-20220822-RI-01 observation end time is outside the LST range
Start time for SCI-20220822-CV-01 is 9.0 and end_time is 11.61
Start time for SCI-20220822-CV-01 is 9.0 and end_time is 11.59
Start time for SCI-20220822-CV-01 is 9.0 and end_time is 11.59
Start time for SCI-20220822-AB-01 is 8.1 and end_time is 13.16
Start time for SCI-20220822-AB-01 is 8.1 and end_time is 13.16
Start time for SCI-20220822-AB-01 is 5.23 and end_time is 10.32
Start time for SCI-20220822-AB-01 is 5.23 and end_time is 10.32
Start time for SCI-20220822-MS-02 is 12.0 and end_time is 15.09
SCI-20220822-MS-02 observation end t

SCI-20220822-MB-01 observation end time is outside the LST range
Start time for SCI-20220822-MB-01 is 6.0 and end_time is 6.0
Start time for SCI-20220822-MB-01 is 12.0 and end_time is 12.0
Start time for SCI-20220822-MB-01 is 8.3 and end_time is 8.3
SCI-20220822-DP-01 observation end time is outside the LST range
SCI-20220822-DP-01 observation end time is outside the LST range
Start time for SCI-20220822-AB-01 is 7.3 and end_time is 12.29
Start time for SCI-20220822-AB-01 is 5.4 and end_time is 10.39
Start time for SCI-20220822-AB-01 is 7.3 and end_time is 12.29
Start time for SCI-20220822-IP-01 is 6.0 and end_time is 11.33
Start time for SCI-20220822-AB-01 is 5.4 and end_time is 10.39
SCI-20220822-HZ-01 both start and the end time are outside the LST range
SCI-20220822-QY-01 observation end time is outside the LST range
SCI-20220822-QY-01 observation end time is outside the LST range
SCI-20220822-HZ-01 observation end time is outside the LST range
Start time for SCI-20220822-MG-01 is 

Start time for SCI-20220822-MB-01 is 8.19 and end_time is 8.19
Start time for SCI-20220822-MB-01 is 8.3 and end_time is 8.3
SCI-20220822-VV-01 both start and the end time are outside the LST range
SCI-20220822-KK-01 observation end time is outside the LST range
SCI-20220822-KK-01 observation end time is outside the LST range
SCI-20220822-KK-01 observation end time is outside the LST range
SCI-20220822-KK-01 observation end time is outside the LST range
SCI-20220822-KK-01 observation end time is outside the LST range
SCI-20220822-KK-01 observation end time is outside the LST range
SCI-20220822-KK-01 observation end time is outside the LST range
SCI-20220822-KK-01 observation end time is outside the LST range
Start time for SCI-20220822-AW-01 is 10.26 and end_time is 10.26
Start time for SCI-20220822-AW-01 is 8.09 and end_time is 12.879999999999999
Date : 2023-03-18 ------------------------------------------------
SCI-20220822-DP-01 observation end time is outside the LST range
SCI-20220

In [17]:

print(ss)
print(sr)

[4.7586684153398044, 4.807289404385963, 4.855670015112881, 4.903818444924977, 4.951742922709037, 4.9994517014218625, 5.0469530761780135, 5.094255396809873, 5.141367074259785, 5.188296582451943, 5.235052443895644, 5.281643240656846, 5.328077612151284, 5.374364272611877, 5.42051203744881, 5.466529822838519, 5.5124266170746905, 5.558211435662323, 5.6038932524795735, 5.649480906446462, 5.69498300613804, 5.740407850744873, 5.7857633737617205, 5.8310571625601915, 5.876296490632869, 5.921488475273886, 5.966640235390174, 6.011759040146292, 6.056852333871874]
[15.912045812939562, 15.990723304031986, 16.069288534597824, 16.14774157386854, 16.226082766913883, 16.304312780288676, 16.382432665890843, 16.460443863323, 16.538348197035912, 16.616147845027076, 16.69384531381672, 16.771443422543932, 16.848945324959054, 16.92635449485296, 17.00367476753653, 17.080910285508327, 17.158065448530547, 17.2351447933945, 17.312152851096773, 17.38909399070793, 17.465972230093364, 17.542791128189776, 17.619553699

In [10]:
# rt = risetime[:,1]
# st = settime[:,1]

# SetRise = nighttime(rt,st)
# #print(len(SetRise))
# for i in SetRise.values():
#     settimes = i[0]
#     risetimes = i[1]
#     names = []
#     for name in obs.keys():
#         names.append(name)
#     d_start = []
#     d_end = []
#     d_added_s = []
#     d_added_e = []
#     for ind, x in enumerate(LST_start):
#         if x > settimes:
#             d_added_s.append([x + duration_[ind], ind])


#         elif LST_end[ind] > settimes:
#              d_added_e.append([LST_end[ind] + duration_[ind], ind])

                
# #Condition 1 based on the LST start time of the observation. 
#     if len(d_added_s) != 0:
#         sub_start = []
    
#         for ind2 in range(len(d_added_s)):
#             if d_added_s[ind2][0] > 24 and (d_added_s[ind2][0]-24) < risetimes:
#                 d_start.append(names[d_added_s[ind2][1]])
#             else:
#                 d_start.append(names[d_added_s[ind2][1]])
#     else:
#         break

# #Condition 2 based on the LST end time of the observation.    
#     if len(d_added_e) != 0:
#         sub_end = []

#         for ind3 in range(len(d_added_e)):
#             if d_added_e[ind3][0] > 24 and d_added_e[ind3][0]-24 < risetimes:
#                 d_end.append(names[d_added_e[ind3][1]])
#             else:
#                 d_end.append(names[d_added_e[ind3][1]])
#     else:
#         break
                          
# for j in obs.keys():
#     if j in d_start:
#         print("{} can run this month with LST start of {} due to condition 1 ".format(j, obs[j][1]))
#     elif j in d_end:
#         print("{} can run this month with LST start of {} due to condition 2".format(j, obs[j][0]))
        

In [11]:
# rt = risetime[:,1]
# st = settime[:,1]
# d_start = []
# d_end = []
# d_added_e = []
# d_added_s = []
# SetRise = nighttime(rt,st)
# #print(len(SetRise))
# for i in SetRise.values():
#     #d_added_s = []
#     settimes = i[0]
#     risetimes = i[1]
#     names = []
#     for name in obs.keys():
#         names.append(name)
#     #d_added_s = []
#     for ind, x in enumerate(LST_start):
#         if settimes - risetimes > 0:
#             if x > settimes and x - settimes > 0:
#                 d_added_s.append([x + duration_[ind], ind])
#             elif 0 < x < risetimes:
#                 d_added_s.append([x + duration_[ind], ind])
#             elif LST_end[ind] > settimes or 0 < LST_end[ind] < risetimes:
#                 d_added_e.append([LST_end[ind] + duration_[ind], ind])


#         elif settimes - risetimes < 0:
#             if x > settimes and 0 < x < risetimes:
#                 d_added_s.append([x + duration_[ind], ind])
#             elif LST_end[ind] > settimes and 0 < x < risetimes:
#                 d_added_s.append([x + duration_[ind], ind])


                
# #Condition 1 based on the LST start time of the observation. 
#     if len(d_added_s) != 0:
#         sub_start = []
    
#         for ind2 in range(len(d_added_s)):
#             if d_added_s[ind2][0] > 24 and (d_added_s[ind2][0]-24) < risetimes:
#                 d_start.append(names[d_added_s[ind2][1]])
#             else:
#                 d_start.append(names[d_added_s[ind2][1]])
#     else:
#         break

# #Condition 2 based on the LST end time of the observation.    
#     if len(d_added_e) != 0:
#         sub_end = []

#         for ind3 in range(len(d_added_e)):
#             if risetimes - d_added_e[ind3][0] >= duration_[ind3]:
#                 if d_added_e[ind3][0] > 24 and (d_added_e[ind3][0]-24) < risetimes:
#                     d_end.append(names[d_added_e[ind3][1]])
#                 else:
#                     d_end.append(names[d_added_e[ind3][1]])
#     else:
#         break
        
        

                          
# for j in obs.keys():
#     if j in d_start:
#         print("{} can run this month with LST start of {} due to condition 1 ".format(j, obs[j][1]))
#     elif j in d_end:
#         print("{} can run this month with LST start of {} due to condition 2".format(j, obs[j][1]))
        